In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

- internet 사용

In [6]:
train = pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip')
test = pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip')
sub = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv')

In [7]:
train.head()

In [8]:
train.info()

In [9]:
train['Sentiment'].value_counts()

타겟값 확인

## SentenceBERT를 사용한 분류

In [1]:
!pip install -q sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer

sentences=["This is an example sentence","Each sentence is converted"]

sbert=SentenceTransformer('sentence-transformers/sentence-t5-base')
embeddings=sbert.encode(sentences)

print(embeddings.shape)

In [10]:
encoded_phrases=sbert.encode(train['Phrase'])

print(encoded_phrases.shape)

In [11]:
encoded_phrases_test=sbert.encode(test['Phrase'])
print(encoded_phrases_test.shape)

In [13]:
from sklearn.model_selection import train_test_split

train_x,val_x,train_y,val_y=train_test_split(
encoded_phrases,train['Sentiment'],test_size=0.2,random_state=42
)

print(train_x.shape,train_y.shape)
print(val_x.shape,val_y.shape)

In [14]:
from keras.models import Sequential
from keras.layers import *

model3=Sequential(
[
    Dense(512,input_shape=(768,),activation='relu'),
    Dropout(0.2),
    Dense(256,activation='relu'),
    Dropout(0.2),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(5,activation='softmax'),
])

model3.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',metrics='acc')

model3.summary()

In [15]:
history3=model3.fit(train_x,train_y,epochs=30,batch_size=1024,validation_data=(val_x,val_y))
result3=model3.predict(encoded_phrases_test)

In [16]:
sub['Sentiment'] = result3.argmax(1)
sub.to_csv('sent3.csv', index=False)